#### Chapter 7: Finetuning To Follow Instructions

In [ ]:
from importlib.metadata import version

pkgs = [
    "numpy",       # PyTorch & TensorFlow dependency
    "matplotlib",  # Plotting library
    "tiktoken",    # Tokenizer
    "torch",       # Deep learning library
    "tqdm",        # Progress bar
    "tensorflow",  # For OpenAI's pretrained weights
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

#### 7.1 Introduction to instruction finetuning
###### In chapter 5, we saw that pretraining an LLM involves a training procedure where it learns to generate one word at a time
###### Hence, a pretrained LLM is good at text completion, but it is not good at following instructions
###### In this chapter, we teach the LLM to follow instructions better
###### The topics covered in this chapter are summarized in the figure below

#### 7.2 Preparing a dataset for supervised instruction finetuning
###### We will work with an instruction dataset I prepared for this chapter

In [ ]:
import json
import os
import urllib


def download_and_load_file(file_path, url):

    if not os.path.exists(file_path):
        with urllib.request.urlopen(url) as response:
            text_data = response.read().decode("utf-8")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text_data)

    # The book originally contained this unnecessary "else" clause:
    #else:
    #    with open(file_path, "r", encoding="utf-8") as file:
    #        text_data = file.read()

    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    return data


file_path = "instruction-data.json"
url = (
    "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch"
    "/main/ch07/01_main-chapter-code/instruction-data.json"
)

data = download_and_load_file(file_path, url)
print("Number of entries:", len(data))

###### Each item in the data list we loaded from the JSON file above is a dictionary in the following form

In [ ]:

print("Example entry:\n", data[50])

###### Note that the 'input' field can be empty:

In [ ]:
print("Another example entry:\n", data[999])

###### Instruction finetuning is often referred to as "supervised instruction finetuning" because it involves training a model on a dataset where the input-output pairs are explicitly provided
###### There are different ways to format the entries as inputs to the LLM; the figure below illustrates two example formats that were used for training the Alpaca (https://crfm.stanford.edu/2023/03/13/alpaca.html) and Phi-3 (https://arxiv.org/abs/2404.14219) LLMs, respectively

###### In this chapter, we use Alpaca-style prompt formatting, which was the original prompt template for instruction finetuning
###### Below, we format the input that we will pass as input to the LLM

In [ ]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text